This notebook set's up an FEM simulation by sampling from the analytical approximation of the surface patch with some $\theta$. Then we translate this into a simnibs input matrix and run a simulation and return the results

In [10]:
import sys

In [11]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/Python_modules/src')

In [12]:
import numpy as np
from simnibs import sim_struct, run_simulation

#For importing geometry library
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

#Timing
from timeit import default_timer as timer

In [13]:
from fieldopt import geolib, tetrapro

In [14]:
#Load up quadratic surface constants, inverse rotation matrix, bounds and normal vector
#This is for the Bayesian optimization component
C = np.fromfile('../../output/quadratic_vec')
iR = np.fromfile('../../output/inverse_rot').reshape((3,3))
b  = np.fromfile('../../output/param_bounds').reshape((3,2))

### Get raw input coordinates that need to be transformed

In [15]:
#Sample a coordinate within the bounds and form input array
x = np.random.uniform(b[0,0], b[0,1])
y = np.random.uniform(b[1,0], b[1,1])

#Sample rotation
r = np.random.uniform(0,180)
print(x,y,r)

(-7.796918002539828, 37.28460237019861, 55.42909125953277)


In [16]:
#Compute z value then map to mesh surface
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Compute orientation vector in basis of surface normal then rotate into FEM mesh space
preaff_rot, preaff_n = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)
n = np.matmul(iR,preaff_n)

In [21]:
#Start a session
S = sim_struct.SESSION()
S.fnamehead="../../data/simnibs_output/sub-CMH090.msh"
S.pathfem="../../data/simnibs_output/test2_simulation"

In [22]:
#Define the TMS simulation
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'

In [23]:
#Define position
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

In [24]:
start = timer()
run_simulation(S)
stop = timer()
print(stop - start)

[ simnibs ]INFO: Could not find the dti tensor file in the standard location: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/d2c_sub-CMH090/dti_results_T1space/DTI_conf_tensor.nii.gz
[ simnibs ]INFO: Head Mesh: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/sub-CMH090.msh
[ simnibs ]INFO: Subject Path: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/m2m_sub-CMH090
[ simnibs ]INFO: Simulation Folder: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test2_simulation
[ simnibs ]INFO: Running simulations in the directory: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test2_simulation
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Using coil: /projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities


0


[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[  1.8715e-01  -5.3941e-01   8.2098e-01  -5.9455e+01]
 [  3.7715e-01  -7.3224e-01  -5.6708e-01   7.1190e+01]
 [ -9.0705e-01  -4.1576e-01  -6.6404e-02   6.7447e+00]
 [  0.0000e+00   0.0000e+00   0.0000e+00   1.0000e+00]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test2_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test2_simulation/tmpOOkjA8.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test2_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_opt

[ simnibs ]INFO: Info    :  45 KSP Residual norm 1.315212511888e-03
[ simnibs ]INFO: Info    :  46 KSP Residual norm 1.095485078065e-03
[ simnibs ]INFO: Info    :  47 KSP Residual norm 8.559523914750e-04
[ simnibs ]INFO: Info    :  48 KSP Residual norm 6.999652191080e-04
[ simnibs ]INFO: Info    :  49 KSP Residual norm 5.584682826268e-04
[ simnibs ]INFO: Info    :  50 KSP Residual norm 4.310198676322e-04
[ simnibs ]INFO: Info    :  51 KSP Residual norm 3.408596362707e-04
[ simnibs ]INFO: Info    :  52 KSP Residual norm 2.784097012078e-04
[ simnibs ]INFO: Info    :  53 KSP Residual norm 2.263020502371e-04
[ simnibs ]INFO: Info    :  54 KSP Residual norm 1.923575015495e-04
[ simnibs ]INFO: Info    :  55 KSP Residual norm 1.747944993283e-04
[ simnibs ]INFO: Info    :  56 KSP Residual norm 1.541364380721e-04
[ simnibs ]INFO: Info    :  57 KSP Residual norm 1.272557644571e-04
[ simnibs ]INFO: Info    :  58 KSP Residual norm 1.026513693846e-04
[ simnibs ]INFO: Info    :  59 KSP Residual norm

292.735257864
